# Warning
This notebook is under development- please use to evaluate entropy notebook and suggest desired changes only.

# Experiments on precision fitting arrays and reality



[Part I: import libraries](#imports)
    
Part II: lookign at EMG versus Drager
    
Part III: squish or not

## Import libraries
<a id= 'imports'> Import libraries for all code</a>



In [1]:

%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import glob
import pandas as pd
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
import collections
from collections import namedtuple
import builtins
import math
import scipy
from scipy.signal import find_peaks
#from TMSiSDK.file_readers import Poly5Reader
# import our library
#sys.path.insert(0, '../resurfemg')
import resurfemg.helper_functions as hf
from pandas import option_context
from resurfemg.config import Config

# config = Config()

In [2]:
sys.path.insert(0, '../resurfemg')
import tsmisdk_lite as tl
#from 

C:\Users\makeda\anaconda3\envs\emgandash\lib\site-packages\mne\fixes.py:321: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(scipy.__version__) >= '1.1':
C:\Users\makeda\anaconda3\envs\emgandash\lib\site-packages\mne\fixes.py:1134: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(numba.__version__) < LooseVersion('0.40'):
C:\Users\makeda\anaconda3\envs\emgandash\lib\site-packages\mne\fixes.py:1134: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(numba.__version__) < LooseVersion('0.40'):


# WARNING!
# The cell below must be eliminated on next release v0.0.4

In [3]:
config = Config()

# Set a collection place for experiments

In [4]:
# do not rerun this cell
big_data_list = []

Below change the path to the root directory where you are keeping your EMGs and ventilator "Draeger" files

# Reruns should be done from this cell as the start

In [5]:
root_emg_directory = config.get_directory('root_emg_directory')

In [6]:
emg_pattern = os.path.join(root_emg_directory, '**/*.Poly5')
emg_and_draeger_files = glob.glob(emg_pattern, recursive=True)

emg_files = []
draeger_files = []

for file in emg_and_draeger_files:
    if 'Draeger' in file:
        draeger_files.append(file)
    else:
        emg_files.append(file)

Now you can pick a file from the list, which have been numbered.

In [7]:
list_of_numbers_strung = []
for i in range(len(emg_files)):
    list_of_numbers_strung.append(str(i))


btn = widgets.Dropdown(
    options=list_of_numbers_strung,
    value='0',
    description='Picked File:',
    disabled=False,
)
display(btn)

Dropdown(description='Picked File:', options=('0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '1…

# Caution! 
If you folder is set up in any way different then the picked file numbers will not neccesarily correspond to the same file. Always check the print out for the file you have chosen in the cell below

In [8]:
number_chosen = int(btn.value)
emg_file_chosen = emg_files[number_chosen]
draeger_file_chosen = draeger_files[number_chosen]
print("The files you chose are:\n", emg_file_chosen, '\n', draeger_file_chosen)

The files you chose are:
 C:/Projects/ReSurfEMG/not_pushed/topspin_data_anonymized\M001\001\EMG_recording.Poly5 
 C:/Projects/ReSurfEMG/not_pushed/topspin_data_anonymized\M001\001\Draeger_recording.Poly5


In [9]:
data_emg = tl.Poly5Reader(emg_file_chosen)
data_samples= data_emg.samples
emg_sample_rate = data_emg.sample_rate
converted_to_seconds =  []
converted_to_samples = []
for i in range(len(data_samples[0])):
    converted_to_seconds.append(i/emg_sample_rate)
    converted_to_samples.append(i)

Reading file  C:/Projects/ReSurfEMG/not_pushed/topspin_data_anonymized\M001\001\EMG_recording.Poly5
	 Number of samples:  172576 
	 Number of channels:  3 
	 Sample rate: 2048 Hz
Done reading data.


In [10]:
data_drag = tl.Poly5Reader(draeger_file_chosen)
data_drag_samples= data_drag.samples
drag_sample_rate =  data_drag.sample_rate
d_converted_to_seconds =  []
d_converted_to_samples = []
for i in range(len(data_drag_samples[0])):
    d_converted_to_seconds.append(i/drag_sample_rate)
    d_converted_to_samples.append(i)

Reading file  C:/Projects/ReSurfEMG/not_pushed/topspin_data_anonymized\M001\001\Draeger_recording.Poly5
	 Number of samples:  8421 
	 Number of channels:  3 
	 Sample rate: 100 Hz
Done reading data.


In [11]:
(len(data_drag_samples[1])/100)- (len(data_samples[1])/2048)

-0.05562500000000625

In [12]:
diff_end_list = []  
emg_file_end_list = []
draeger_file_end_list = []
for num in list(range(25,35)): #len(emg_files))):
          
    number_chosen = int(num)
    emg_file_chosen = emg_files[number_chosen]
    draeger_file_chosen = draeger_files[number_chosen]
    data_drag = tl.Poly5Reader(draeger_file_chosen)
    data_drag_samples= data_drag.samples
    data_emg = tl.Poly5Reader(emg_file_chosen)
    data_samples= data_emg.samples
    longer = (len(data_drag_samples[1])/100) - (len(data_samples[1])/2048)
    diff_end_list.append(longer)
    emg_file_end_list.append(emg_file_chosen) 
    draeger_file_end_list.append(draeger_file_chosen) 
end_list = pd.DataFrame([emg_file_end_list, draeger_file_end_list, diff_end_list]) 
end_list = end_list.T
end_list.columns = ['emg','draeg', 'diff']
end_list

Reading file  C:/Projects/ReSurfEMG/not_pushed/topspin_data_anonymized\M003\007\Draeger_recording.Poly5
	 Number of samples:  36490 
	 Number of channels:  3 
	 Sample rate: 100 Hz
Done reading data.
Reading file  C:/Projects/ReSurfEMG/not_pushed/topspin_data_anonymized\M003\007\EMG_recording.Poly5
	 Number of samples:  747360 
	 Number of channels:  3 
	 Sample rate: 2048 Hz
Done reading data.
Reading file  C:/Projects/ReSurfEMG/not_pushed/topspin_data_anonymized\M003\008\Draeger_recording.Poly5
	 Number of samples:  30841 
	 Number of channels:  3 
	 Sample rate: 100 Hz
Done reading data.
Reading file  C:/Projects/ReSurfEMG/not_pushed/topspin_data_anonymized\M003\008\EMG_recording.Poly5
	 Number of samples:  631504 
	 Number of channels:  3 
	 Sample rate: 2048 Hz
Done reading data.
Reading file  C:/Projects/ReSurfEMG/not_pushed/topspin_data_anonymized\M003\009\Draeger_recording.Poly5
	 Number of samples:  38903 
	 Number of channels:  3 
	 Sample rate: 100 Hz
Done reading data.
Read

,emg,draeg,diff
0,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,-0.021875
1,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,0.058438
2,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,-0.0325
3,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,-0.065938
4,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,-0.121875
5,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,-0.235312
6,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,-0.14875
7,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,-0.117813
8,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,-0.184062
9,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,-0.097188


In [14]:
diff_start_list = []  
emg_file_start_list = []
draeger_file_start_list = []
for num in list(range(24)): #len(emg_files))):
          
    number_chosen = int(num)
    emg_file_chosen = emg_files[number_chosen]
    draeger_file_chosen = draeger_files[number_chosen]
    data_drag = tl.Poly5Reader(draeger_file_chosen)
    data_drag_samples= data_drag.samples
    data_emg = tl.Poly5Reader(emg_file_chosen)
    data_samples= data_emg.samples
    longer = (len(data_drag_samples[1])/100) - (len(data_samples[1])/2048)
    diff_start_list.append(longer)
    emg_file_start_list.append(emg_file_chosen) 
    draeger_file_start_list.append(draeger_file_chosen)
start_list = pd.DataFrame([emg_file_start_list, draeger_file_start_list, diff_start_list]) 
start_list = start_list.T
start_list.columns = ['emg','draeg', 'diff']
start_list

Reading file  C:/Projects/ReSurfEMG/not_pushed/topspin_data_anonymized\M001\001\Draeger_recording.Poly5
	 Number of samples:  8421 
	 Number of channels:  3 
	 Sample rate: 100 Hz
Done reading data.
Reading file  C:/Projects/ReSurfEMG/not_pushed/topspin_data_anonymized\M001\001\EMG_recording.Poly5
	 Number of samples:  172576 
	 Number of channels:  3 
	 Sample rate: 2048 Hz
Done reading data.
Reading file  C:/Projects/ReSurfEMG/not_pushed/topspin_data_anonymized\M001\002\Draeger_recording.Poly5
	 Number of samples:  48746 
	 Number of channels:  3 
	 Sample rate: 100 Hz
Done reading data.
Reading file  C:/Projects/ReSurfEMG/not_pushed/topspin_data_anonymized\M001\002\EMG_recording.Poly5
	 Number of samples:  998496 
	 Number of channels:  3 
	 Sample rate: 2048 Hz
Done reading data.
Reading file  C:/Projects/ReSurfEMG/not_pushed/topspin_data_anonymized\M001\003\Draeger_recording.Poly5
	 Number of samples:  42449 
	 Number of channels:  3 
	 Sample rate: 100 Hz
Done reading data.
Readi

Done reading data.
Reading file  C:/Projects/ReSurfEMG/not_pushed/topspin_data_anonymized\M003\003\Draeger_recording.Poly5
	 Number of samples:  44374 
	 Number of channels:  3 
	 Sample rate: 100 Hz
Done reading data.
Reading file  C:/Projects/ReSurfEMG/not_pushed/topspin_data_anonymized\M003\003\EMG_recording.Poly5
	 Number of samples:  909104 
	 Number of channels:  3 
	 Sample rate: 2048 Hz
Done reading data.
Reading file  C:/Projects/ReSurfEMG/not_pushed/topspin_data_anonymized\M003\004\Draeger_recording.Poly5
	 Number of samples:  38267 
	 Number of channels:  3 
	 Sample rate: 100 Hz
Done reading data.
Reading file  C:/Projects/ReSurfEMG/not_pushed/topspin_data_anonymized\M003\004\EMG_recording.Poly5
	 Number of samples:  783952 
	 Number of channels:  3 
	 Sample rate: 2048 Hz
Done reading data.
Reading file  C:/Projects/ReSurfEMG/not_pushed/topspin_data_anonymized\M003\005\Draeger_recording.Poly5
	 Number of samples:  41605 
	 Number of channels:  3 
	 Sample rate: 100 Hz
Done

,emg,draeg,diff
0,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,-0.055625
1,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,-0.086875
2,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,0.036875
3,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,-0.105312
4,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,-0.030312
5,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,-0.0025
6,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,-0.062187
7,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,0.029688
8,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,-0.03875
9,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,C:/Projects/ReSurfEMG/not_pushed/topspin_data_...,-0.23625


In [ ]:
full_list = pd.concat([end_list, start_list])
full_list.to_csv('list_of_time_differences.csv')

In [ ]:
#full_list.style.set_properties(subset=['emg','draeg'], **{'width': '300px'})


with option_context('display.max_colwidth', 400):
    display(full_list)

#df.style.set_properties(subset=['text'], **{'width': '300px'})

In [ ]:
#print(diff_list)

Part III:  the new data is it squeezed?

Let's import something we know has an exactly 22 respiratory rate!

In [ ]:
d22_chosen = 'C:/Projects/ReSurfEMG/not_pushed/d22/Draeger_recording_22.Poly5'
data_vent_22 = tl.Poly5Reader(d22_chosen)
data_samples= data_vent_22.samples
data_sample_rate = data_vent_22.sample_rate
converted_to_seconds =  []
converted_to_samples = []
for i in range(len(data_samples[0])):
    converted_to_seconds.append(i/data_sample_rate)
    converted_to_samples.append(i)

In [ ]:
%matplotlib inline
# set up plotn
x = data_samples
fig, axis = plt.subplots(nrows = 3, ncols = 2, figsize=(16, 6))

axis[0,0].grid(True)
axis[0,0].plot(x[0])
axis[0,0].set(title='leads in samples')
axis[1,0].plot(x[1])
axis[2,0].plot(x[2])
## below needs a rewrite
axis[0,1].set(title='leads in seconds')
axis[0,1].grid(True)
axis[0,1].plot(converted_to_seconds,x[0])
axis[1,1].plot(converted_to_seconds,x[1])
axis[2,1].plot(converted_to_seconds,x[2])

In [ ]:
plt.plot(converted_to_seconds[0:6000],x[0][0:6000])


In [ ]:
plt.plot(converted_to_seconds[6000:12000],x[0][6000:12000])

In [ ]:
22/60

In [ ]:
def find_ventilator_rate(vent_signal_file):
    file_name = vent_signal_file
    data_drag = tl.Poly5Reader(vent_signal_file)
    data_drag_samples= data_drag.samples
    data_drag_sample_rate = data_drag.sample_rate
    #print(data_drag_samples)
    peaks, _ = find_peaks(data_drag_samples[0], distance =1.9*data_drag_sample_rate,  height=0)
    plt.plot(data_drag_samples[0])
    plt.plot(peaks, data_drag_samples[0][peaks], "x")
    plt.rcParams["figure.figsize"] = (2,3)
    rate = (len(peaks)/(len(data_drag_samples[0])/100))*60
    time = (len(data_drag_samples[0])/100)/60
    return rate, time, file_name

In [ ]:
num_list = []
rate_list = []
time_list = []
file_list = []
for num in list(range(24)):
    num_list.append( num)
    rate_list.append(find_ventilator_rate(draeger_files[num])[0])
    time_list.append(find_ventilator_rate(draeger_files[num])[1])
    file_list.append(find_ventilator_rate(draeger_files[num])[2])

In [ ]:
rate_df =pd.DataFrame(num_list, columns = ['num'])
rate_df['rate'] = rate_list
rate_df['time'] = time_list
rate_df['file'] = file_list
rate_df

In [ ]:
rate_df.to_csv('rates_by_file.csv')

In [ ]:
%matplotlib inline
# set up plotn
x = data_samples
fig, axis = plt.subplots(nrows = 3, ncols = 2, figsize=(6, 6))
#ax.set_ylim([-4, 4])
axis[0,0].grid(True)
axis[0,0].plot(x[0])
axis[0,0].set(title='leads in samples')
axis[1,0].plot(x[1])
axis[2,0].plot(x[2])
## needs a rewrite
# axis[0,1].set(title='leads in seconds')
# axis[0,1].grid(True)
# axis[0,1].plot(converted_to_seconds,x[0])
# axis[1,1].plot(converted_to_seconds,x[1])
# axis[2,1].plot(converted_to_seconds,x[2])

In [ ]:
%matplotlib inline
# set up plotn
z = data_samples[2]
# fig, axis = plt.subplots(nrows = 1, ncols = 1, figsize=(16, 6))
# #ax.set_ylim([-4, 4])
# axis.grid(True)
# axis.plot(x[0])
# axis.set(title='leads in samples')
plt.plot(z[:20000])


In [ ]:
z_by_bad_end_cut = hf.bad_end_cutter_for_samples(
        data_samples,
        percent_to_cut=3,
        tolerance_percent=5
    )
z2 = z_by_bad_end_cut[2]

plt.plot(z2)
plt.plot(z[:169478], alpha = 0.4)

In [ ]:
#plt.plot(z2-z[:169478], alpha = 0.4)

In [ ]:
 bd_filtered_file_data = hf.emg_bandpass_butter_sample(
        z_by_bad_end_cut,
        5,
        450,
        2048,
        output='sos'
    )

In [ ]:
plt.plot(z2)
plt.plot(bd_filtered_file_data[2])


In [ ]:
print(len(z2))
print(len(bd_filtered_file_data[0]))

In [ ]:
z3_3 = hf.bad_end_cutter_for_samples(
        bd_filtered_file_data,
        percent_to_cut=3,
        tolerance_percent=5
    )

In [ ]:
plt.plot(bd_filtered_file_data[0][140000:150000])
plt.plot(z3_3[2][140000:150000], alpha=0.5)


In [ ]:

components = hf.compute_ICA_two_comp(z3_3)

emg = hf.pick_lowest_correlation_array(components, z3_3[0])
print(len(emg))
print(len(z3_3[1]))

In [ ]:
plt.plot(emg[140000:150000]*10000)
plt.plot(z3_3[2][140000:150000], alpha=0.5)


In [ ]:

componentsM = hf.compute_ICA_two_comp_multi(z3_3)

emgM = hf.pick_lowest_correlation_array(componentsM, z3_3[0])
print(len(emgM))
print(len(z3_3[1]))

In [ ]:
plt.plot(emgM[140000:150000]*10000)
plt.plot(z3_3[2][140000:150000], alpha=0.5)


In [ ]:
plt.plot(emgM[140000:150000]*10000)
plt.plot(emg[140000:150000]*10000, alpha=0.4)

In [ ]:
plt.plot(emg*1000)

In [ ]:
# now let's pre-process

def working_pipeline_pre_ml(our_chosen_samples, picker='heart'):
    """
    :param our_chosen_samples: the read EMG file arrays
    :type our_chosen_samples: ~numpy.ndarray
    :param picker: the picking strategy for independant components
    :type picker: str

    :returns: final_envelope_a
    :rtype: ~numpy.ndarray
    """
    cut_file_data = bad_end_cutter_for_samples(
        our_chosen_samples,
        percent_to_cut=3,
        tolerance_percent=5
    )
    bd_filtered_file_data = emg_bandpass_butter_sample(
        cut_file_data,
        5,
        450,
        2048,
        output='sos'
    )
    # step for end-cutting again to get rid of filtering artifacts
    re_cut_file_data = bad_end_cutter_for_samples(
        bd_filtered_file_data,
        percent_to_cut=3,
        tolerance_percent=5
    )
    #  and do step for ICA
    components = compute_ICA_two_comp(re_cut_file_data)
    #     the picking step!
    if picker == 'peaks':
        emg = pick_more_peaks_array(components)
    elif picker == 'heart':
        emg = pick_lowest_correlation_array(components, re_cut_file_data[0])
    else:
        emg = pick_lowest_correlation_array(components, re_cut_file_data[0])
        print("Please choose an exising picker i.e. peaks or hearts ")
    # now process it in final steps
    abs_values = abs(emg)
    final_envelope_d = emg_highpass_butter(abs_values, 150, 2048)

    return final_envelope_d

In [ ]:
%matplotlib inline
# set up plotn
x = x[:, 20000:40000]
fig, axis = plt.subplots(nrows = 3, ncols = 1, figsize=(16, 6))
#ax.set_ylim([-4, 4])
axis[0].grid(True)
axis[0].plot(x[0])
axis[0].set(title='leads in samples')
axis[1].plot(x[1])
axis[2].plot(x[2])


In [ ]:
len(data_samples[0])

In [ ]:
len(data_samples[0])/2048

In [ ]:
(len(data_samples[0])/2048)/60

That is the whole unfiltered EMG, number 0. Let's take the real legnth, and sample rate, and double check time with ROB

Times on each sample: 
\M001\001\EMG_recording.Poly5  should be about 5 minutes
\M001\001\EMG_recording.Poly5
 stable
002-005 5 to 10 minutes

006-009 5 to 10 minute, at the end each should have 3 occlusion events

Sample rates on each sample: 
2048 all

Known # breaths:


----





Do you want to cut and see the file in samples or seconds?

In [ ]:
y_axis = widgets.Dropdown(
    options=["Samples","Seconds"],
    value='Samples',
    description="Select View Option",
    disabled=False,
)
display(y_axis)


In [ ]:
time_view= y_axis.value
time_view

Now you will pick the start and end of your sample.We are going to clip the end of the sample in processing, so you can not pick any values and get a good graph. We preset the values towards the max graphable with ease, but they can be overwritten.  In the future we will have an updating graph here. 

In [ ]:
if time_view == 'Samples':
    int_slider1 = widgets.IntSlider(
        min=0, max=int(len(x[0])*0.89), step=1,
        description=' samples start'
    )
    int_slider2 = widgets.IntSlider(
        value=len(x[0]),
        min=0, max=int(len(x[0])*0.89), step=1,
        description='samples end cutoff'
    )
else:
    int_slider1 = widgets.IntSlider(
        #value=0.1,
        min=0, max= int(converted_to_seconds[-1])*0.89, step=1,
        description='seconds start'
    )
    int_slider2 = widgets.IntSlider(
        #value=converted_to_seconds[-1],
        min=0, max=int(converted_to_seconds[-1])*0.89, step= 1,
        description='seconds end cutoff'
    )
    
widgets.VBox(
    [

        int_slider1,
        int_slider2,

    ]
)



You can overwrite the values by hand in the next cell, if scrolling is not precise enough...but rewriting to take an absolute end is unadvisable.

In [ ]:
# # # # Here we can overwrite the values by hand, again you must pick values a bit inside
# int_slider1.value = 1
# int_slider2.value = 17

For now we will graph your choice in the next active cell

In [ ]:
start = int_slider1.value
end= int_slider2.value
if time_view == 'Samples':
    # nox examine at a certain scale- from point a to b as samples
    x = data_samples
    fig, (ax_1,ax_2,ax_3) = plt.subplots(nrows = 3, figsize=(6, 4))
    ax_1.grid(True)
    ax_1.plot(x[0][int(start):int(end)])
    ax_1.set(title='leads, samples')
    ax_2.plot(x[1][int(start):int(end)])
    ax_3.plot(x[2][int(start):int(end)])
    
if time_view == 'Seconds':
    # nox examine at a certain scale- from point a to b as samples
    x_for_secs = data_samples

    y = converted_to_seconds
    fig, (ax_1,ax_2,ax_3) = plt.subplots(nrows = 3, figsize=(6, 4))
    ax_1.grid(True)
    ax_1.plot(y[int(start*emg_sample_rate):int(end*emg_sample_rate)],x[0][int(start*emg_sample_rate):int(end*emg_sample_rate)])
    ax_1.set(title='leads, seconds')
    ax_2.plot(y[int(start*emg_sample_rate):int(end*emg_sample_rate)],x[1][int(start*emg_sample_rate):int(end*emg_sample_rate)])
    ax_3.plot(y[int(start*emg_sample_rate):int(end*emg_sample_rate)],x[2][int(start*emg_sample_rate):int(end*emg_sample_rate)])
    

Happy with your selection? If not redo the widgeted cell, then we can see how the filter the selection in a basic pipleline before extracting entropy.

In [ ]:
# dropdown to pick ICA possibilities. CUrrently only one -\o/-
ICA_choice = widgets.Dropdown(
    options=["classic","no_ica_lead3"],
    value='classic',
    description="Select View Option",
    disabled=False,
)
display(ICA_choice)

Here we will have to rewrite to accomodate different ICAs, but this is in the future. After we iron out the alternative ICAs. Below we put our EMG data through the pipeline we have now, and we must do picking from an ICA by more peaks or by dis-similarity to the heart/ECG lead.

In [ ]:
ICA_picker_choice = widgets.Dropdown(
    options=["more_peaks","similar_to_ECG"],
    value='more_peaks',
    description="Select View Option",
    disabled=False,
)
display(ICA_picker_choice)

In [ ]:
ICA_picker_choice.value

In [ ]:
def working_pipeline_pre_entropy_peaks(our_chosen_samples): 
    cut_file_data = hf.bad_end_cutter_for_samples(our_chosen_samples, percent_to_cut=3, tolerance_percent=5)
    bd_filtered_file_data = hf.emg_bandpass_butter_sample(cut_file_data, 5, 450, 2048, output='sos')
    # step 3 end-cutting again to get rid of filtering artifacts
    re_cut_file_data = hf.bad_end_cutter_for_samples(bd_filtered_file_data, percent_to_cut=3, tolerance_percent=5)
    # skip step4 and do step 5 ICA
    components = hf.compute_ICA_two_comp(re_cut_file_data)
    #     the picking step!
    emg= hf.pick_more_peaks_array(components)
    # now process it in final steps
    abs_values = abs(emg)
    final_envelope_d = hf.emg_highpass_butter(abs_values, 150, 2048)
    
        
    return final_envelope_d

def working_pipeline_pre_entropy_ecg(our_chosen_samples): 
    cut_file_data = hf.bad_end_cutter_for_samples(our_chosen_samples, percent_to_cut=3, tolerance_percent=5)
    bd_filtered_file_data = hf.emg_bandpass_butter_sample(cut_file_data, 5, 450, 2048, output='sos')
    # step 3 end-cutting again to get rid of filtering artifacts
    re_cut_file_data = hf.bad_end_cutter_for_samples(bd_filtered_file_data, percent_to_cut=3, tolerance_percent=5)
    # skip step4 and do step 5 ICA
    components = hf.compute_ICA_two_comp(re_cut_file_data)
    #     the picking step!
    emg= hf.pick_lowest_correlation_array(components,re_cut_file_data[0] )
    # now process it in final steps
    abs_values = abs(emg)
    final_envelope_d = hf.emg_highpass_butter(abs_values, 150, 2048)
    
        
    return final_envelope_d

# Let's examine our processed EMG

In [ ]:
if ICA_picker_choice.value == 'more_peaks':
    processed_data_emg = working_pipeline_pre_entropy_peaks(data_samples)
elif ICA_picker_choice.value == 'similar_to_ECG':
    processed_data_emg = working_pipeline_pre_entropy_ecg(data_samples)
else:
    processed_data_emg = working_pipeline_pre_entropy_ecg(data_samples)

if time_view == 'Seconds':
    %matplotlib inline
    # set up plotn
    x = processed_data_emg
    fig, axis = plt.subplots(nrows = 1, ncols = 2, figsize=(14, 6))
    axis[0].grid(True)
    axis[0].plot(converted_to_seconds[:len(x)], x)
    axis[0].set(title='The whole sample minus last filter lost end')
    axis[1].set(title='Your picked area in seconds')
    axis[1].grid(True)
    axis[1].plot(converted_to_seconds[int(start*emg_sample_rate):int(end*emg_sample_rate)],x[int(start*emg_sample_rate):int(end*emg_sample_rate)])
else:
    %matplotlib inline
    x = processed_data_emg
    fig, axis = plt.subplots(nrows = 1, ncols = 2, figsize=(14, 6))
    axis[0].grid(True)
    axis[0].plot(x)
    axis[0].set(title='The whole sample minus last filter lost end')
    axis[1].set(title='Your picked area in samples')
    axis[1].grid(True)
    axis[1].plot(converted_to_samples[int(start):int(end)],x[int(start):int(end)])
    

Now we created some basic processed EMG. We will graph it based on the sample selected and the cutoff on entropy. We will ultimately do a cut-off based on something popular in the literature, but let's do one based on one simple parameter first, as an example


9 breaths per 40,000 samples

In [ ]:
40000/2048

In [ ]:
(9/19.53125)*60

In [ ]:
(8/19.53125)*60

In [ ]:
# Now we want to select where the cut_off is

entropy_cutoff = widgets.Dropdown(
    options=["Mean","Half_range"],
    value='Mean',
    description="Select Entropy Cut off",
    disabled=False,
)
display(entropy_cutoff)


In [ ]:
def rounded_for_ent(stralist):
    rounded= np.round_(stralist, decimals = 5)
    return rounded
start_s= start * emg_sample_rate
end_s = end * emg_sample_rate
if time_view == 'Samples':
    big_list = rounded_for_ent(processed_data_emg[int(start):int(end)])# replace with whole array of time series!
else:
   
    big_list = rounded_for_ent(processed_data_emg[int(start_s):int(end_s)])
slice_length = 100
def sliceIterator(lst, sliceLen):
    for i in range(len(lst) - sliceLen + 1):
        yield lst[i:i + sliceLen]
index_hold = []
for slice in sliceIterator(big_list, slice_length):
    entropy_index = hf.entropical(slice)
    index_hold.append(entropy_index)

if entropy_cutoff.value == 'Half_range':
    decision_cutoff = (np.max(index_hold) + np.min(index_hold))/2
else:# entropy_cutoff.value == 'Mean':
    decision_cutoff = np.mean(index_hold)


rms_rolled = hf.vect_naive_rolling_rms(index_hold,100) # so rms is rms entropy

Now let's see how we would have split it on that criteria

In [ ]:
if time_view == 'Samples':
    #y= converted_to_samples
    plt.plot(converted_to_samples[int(start):(int(start) + len(rms_rolled))],processed_data_emg[int(start):(int(start) + len(rms_rolled))]*1000)
    plt.plot(converted_to_samples[int(start):(int(start) + len(rms_rolled))],rms_rolled)
    plt.plot(converted_to_samples[int(start):(int(start) + len(rms_rolled))],hf.zero_one_for_jumps_base(rms_rolled,decision_cutoff))
    plt.axhline(y = decision_cutoff, color = 'r', linestyle = '-')
    
else:
    y = converted_to_seconds
    plt.plot(converted_to_seconds[int(start_s):(int(start_s)+int(len(rms_rolled)))], processed_data_emg[int(start_s):(int(start_s)+len(rms_rolled))]*1000)
    plt.plot(converted_to_seconds[int(start_s):(int(start_s)+int(len(rms_rolled)))],rms_rolled)
    plt.plot(converted_to_seconds[int(start_s):(int(start_s)+int(len(rms_rolled)))],hf.zero_one_for_jumps_base(rms_rolled,decision_cutoff))
    plt.axhline(y = decision_cutoff, color = 'r', linestyle = '-')
    

In the above picture the green line represents a 0,1, array which represents the breaths. That picking was based on one simple parameter.

# Changes here!
Instead of above code we will do
pick breath based on 90% entropy
then 50%- this will give start of breath
define peak value within 90%
moving forward to right 70%-> with smoothing away if there is too short a pause. This is a more complicated algorithm. 

In [ ]:
# reminder
time_view

In [ ]:

def rounded_for_ent(stralist):
    rounded= np.round_(stralist, decimals = 5)
    return rounded
if time_view == 'Samples':
    big_list = rounded_for_ent(processed_data_emg[int(start):int(end)])# replace with whole array of time series!
else:
    big_list = rounded_for_ent(processed_data_emg[int(start_s):int(end_s)])
slice_length = 100
def sliceIterator(lst, sliceLen):
    for i in range(len(lst) - sliceLen + 1):
        yield lst[i:i + sliceLen]
index_hold = []
for slice in sliceIterator(big_list, slice_length):
    entropy_index = hf.entropical(slice)
    index_hold.append(entropy_index)

high_decision_cutoff = 0.9  * ((np.max(index_hold)) - (np.min(index_hold))) + np.min(index_hold)
decision_cutoff = 0.5 * ((np.max(index_hold)) - (np.min(index_hold))) + np.min(index_hold)

rms_rolled = hf.vect_naive_rolling_rms(index_hold,100) # so rms is rms entropy


if time_view == 'Samples':
    plt.plot(converted_to_samples[int(start):(int(start) + len(rms_rolled))],processed_data_emg[int(start):(int(start) + len(rms_rolled))]*1000, alpha = 0.5)
    plt.plot(converted_to_samples[int(start):(int(start) + len(rms_rolled))],rms_rolled)
    plt.plot(converted_to_samples[int(start):(int(start) + len(rms_rolled))],hf.zero_one_for_jumps_base(rms_rolled,decision_cutoff))
    plt.plot(converted_to_samples[int(start):(int(start) + len(rms_rolled))],hf.zero_one_for_jumps_base(rms_rolled,high_decision_cutoff), color= 'purple')
    plt.axhline(y = decision_cutoff, color = 'r', linestyle = '-')
    plt.axhline(y = high_decision_cutoff, color = 'purple', linestyle = '-')
    
else:
    y = converted_to_seconds
    plt.plot(converted_to_seconds[int(start_s):(int(start_s)+int(len(rms_rolled)))], processed_data_emg[int(start_s):(int(start_s)+len(rms_rolled))]*1000, alpha = 0.5)
    plt.plot(converted_to_seconds[int(start_s):(int(start_s)+int(len(rms_rolled)))],rms_rolled)
    plt.plot(converted_to_seconds[int(start_s):(int(start_s)+int(len(rms_rolled)))],hf.zero_one_for_jumps_base(rms_rolled,decision_cutoff))
    plt.plot(converted_to_seconds[int(start_s):(int(start_s)+int(len(rms_rolled)))],hf.zero_one_for_jumps_base(rms_rolled,high_decision_cutoff), color = 'purple')
    plt.axhline(y = decision_cutoff, color = 'r', linestyle = '-')
    plt.axhline(y = high_decision_cutoff, color = 'purple', linestyle = '-')
    

In [ ]:
def merge(left, right):
    # Initialize an empty list output that will be populated with sorted elements.
    # Initialize two variables i and j which are used pointers when iterating through the lists.
    output = []
    i = j = 0

    # Executes the while loop if both pointers i and j are less than the length of the left and right lists
    while i < len(left) and j < len(right):
        # Compare the elements at every position of both lists during each iteration
        if left[i] < right[j]:
            # output is populated with the lesser value
            output.append(left[i])
            # 10. Move pointer to the right
            i += 1
        else:
            output.append(right[j])
            j += 1
    # The remnant elements are picked from the current pointer value to the end of the respective list
    output.extend(left[i:])
    output.extend(right[j:])

    return output

In [ ]:
merge

In [ ]:


slice = builtins.slice

class Range(namedtuple('RangeBase', 'start,end')):
    
    def intersects(self, other):
        return (
            (self.end >= other.end) and (self.start < other.end) or
            (self.end >= other.start) and (self.start < other.start) or
            (self.end < other.end) and (self.start >= other.start)
        )
    
    def precedes(self, other):
        return self.end < other.start
    
    def to_slice(self):
        return slice(*map(int, self)) # maps whole tuple set
    


def ranges_of(array):
    marks = np.logical_xor(array[1:], array[:-1])
    boundaries = np.hstack((np.zeros(1), np.where(marks != 0)[0], np.zeros(1) + len(array) - 1))

    if not array[0]:
        boundaries = boundaries[1:]
    if len(boundaries) % 2 != 0:
        boundaries = boundaries[:-1]
    return tuple(Range(*boundaries[i:i+2]) for i in range(0, len(boundaries), 2))


def intersections(left, right):
    i, j = 0, 0
    result = []
    while i < len(left) and j < len(right):
        lelt, relt = left[i], right[j]
        if lelt.intersects(relt):
            result.append(lelt)
            i += 1
        elif relt.precedes(lelt):
            j += 1
        elif lelt.precedes(relt):
            i += 1
    return result


hi = np.array(hf.zero_one_for_jumps_base(rms_rolled, high_decision_cutoff))
lo = np.array(hf.zero_one_for_jumps_base(rms_rolled, decision_cutoff))

rhi = ranges_of(hi)
rlo = ranges_of(lo)

keep = intersections(rlo, rhi)


points = np.array(sum(keep, start=()), dtype=np.int32)
points

In [ ]:
seven_line = np.zeros(len(rms_rolled))
for seven_range in keep:
    seven_line[seven_range.to_slice()] = 7
if time_view == 'Samples':
    plt.plot(converted_to_samples[int(start):(int(start) + len(rms_rolled))],processed_data_emg[int(start):(int(start) + len(rms_rolled))]*1000, alpha = 0.5)
    plt.plot(converted_to_samples[int(start):(int(start) + len(rms_rolled))],hf.zero_one_for_jumps_base(rms_rolled,decision_cutoff),  color = 'green')
    plt.plot(converted_to_samples[int(start):(int(start) + len(rms_rolled))],(np.array(hf.zero_one_for_jumps_base(rms_rolled,high_decision_cutoff)))*2, color= 'purple')
    #plt.plot(converted_to_samples[int(start):(int(start) + len(rms_rolled))], six_line)
    plt.plot(converted_to_samples[int(start):(int(start) + len(rms_rolled))], seven_line)
else:
    plt.plot(converted_to_seconds[int(start_s):(int(start_s) + len(rms_rolled))],processed_data_emg[int(start_s):(int(start_s) + len(rms_rolled))]*1000, alpha = 0.5)
    plt.plot(converted_to_seconds[int(start_s):(int(start_s) + len(rms_rolled))],hf.zero_one_for_jumps_base(rms_rolled,decision_cutoff),  color = 'green')
    plt.plot(converted_to_seconds[int(start_s):(int(start_s) + len(rms_rolled))],(np.array(hf.zero_one_for_jumps_base(rms_rolled,high_decision_cutoff)))*2, color= 'purple')
    #plt.plot(converted_tosecondss[int(star_st):(int(star_st) + len(rms_rolled))], six_line)
    plt.plot(converted_to_seconds[int(start_s):(int(start_s) + len(rms_rolled))], seven_line)
    

So above our 'seven-line' represents picking based on finding areas with 90% max entropy, then picking evything around them with over 50% entropy.

## What happened in these breaths?

We want to know the area under the curve for each breath- then the area to where we cross back over 70% of peak values- but should we look at entropy directly- question for Eline. Let's start with breaking the segments into breath or not breath segments.

In [ ]:
# our_emg_array = processed_data_emg[int(start):(int(start) + len(rms_rolled))]
# jump_indeces = []
# zippy = zip(seven_line,seven_line[1:])
# for val in enumerate(zippy):
#     if val[1][0] != val[1][1]:
#         print(val[0])
#         jump_indeces.append(val[0])

# grouped = np.split(our_emg_array, jump_indeces)
# grouped

Now we need to figure out which elements of grouped are breath parts, and make arrays of just those.

In [ ]:
# so grouped needs to be picked down to where at jump indeces we go up from 0 to 7
our_emg_array_samples = processed_data_emg[int(start):(int(start) + len(rms_rolled))]
our_emg_array_seconds = processed_data_emg[int(start_s):(int(start_s) + len(rms_rolled))]
zippy = zip(seven_line,seven_line[1:])
breath_indeces = []
for val in enumerate(zippy):
    if val[1][0] < val[1][1]:
        print(val[0])
        breath_indeces.append(val[0])
if time_view == 'Samples':
    grouped_breaths = np.split(our_emg_array_samples, breath_indeces)
    grouped_entropy= np.split(rms_rolled, breath_indeces)

else:
    grouped_breaths = np.split(our_emg_array_seconds, breath_indeces)
    grouped_entropy= np.split(rms_rolled, breath_indeces)

In [ ]:
%matplotlib widget
for group in grouped_breaths:
    group = abs(group)
    plt.plot(group, alpha = 0.5)


In [ ]:
%matplotlib widget
for group in grouped_entropy:
     plt.plot(group, alpha = 0.5)

# now for each breath started segment get the "outside peak envelope", smooth it, and get the area under the curve until we reach 70%
The first cell below is an examiner where we plot out the absolute value on each breath group to make sure we are catching breaths.

Now we will plot out the smoothed breaths:

In [ ]:
for group in grouped_breaths:
    group = abs(group)

In [ ]:
%matplotlib inline

plt.plot(abs(grouped_breaths[3]))
plt.show()

In [ ]:
len(x)

In [ ]:
x = abs(grouped_breaths[3])

peaks, _ = find_peaks(x, distance =len(x)/50,  height=0)

plt.plot(x)

plt.plot(peaks, x[peaks], "x")

plt.plot(np.zeros_like(x), "--", color="gray")

plt.show()

In [ ]:

def hi_envelope(signal, dmax=24):
    """
    Takes a 1d signal array, and extracts 'high'envelope, then makes high envelope
    dmax: int, ize of chunks, 
    
    """
    # locals max
    lmax = (np.diff(np.sign(np.diff(signal))) < 0).nonzero()[0] + 1 
    lmax = lmax[[i+np.argmax(s[lmax[i:i+dmax]]) for i in range(0,len(lmax),dmax)]]
    smoothed = savgol_filter(signal[high_idx] , int(0.8* (len(high_idx ))), 3)
    smoothed_interped = scipy.signal.resample(smoothed, len(signal))
    return smoothed_interped 

In [ ]:
smoothed = savgol_filter(signal[high_idx] , int(0.8* (len(high_idx ))), 3)
smoothed_interped = scipy.signal.resample(smoothed, len(grouped_breaths[3]))

In [ ]:
len(high_idx)

In [ ]:
len(smoothed)

In [ ]:
len(smoothed_interped)

In [ ]:
s

In [ ]:
smoothed

In [ ]:
t = np.linspace(0,len(grouped_breaths[3]),len(grouped_breaths[3]))
s = abs(grouped_breaths[3])
#high_idx, low_idx = hl_envelopes_idx(s)

# high_idx = hi_envelope_points(s,dmax=24)
# smoothed = savgol_filter(s[high_idx] , int(0.8* (len(high_idx ))), 3)
# smoothed_interped = scipy.signal.resample(smoothed, len(grouped_breaths[3]))
smoothed_i = hi_envelope(s,dmax=24)
# plot
plt.plot(t,s,label='signal')
#plt.plot(t[high_idx],s[high_idx], 'r', label='low')
plt.plot(t,smoothed_i, 'g', label='high')
plt.show()

In [ ]:
t = np.linspace(0,8*np.pi,5000)
s = 0.8*np.cos(t)**3 + 0.5*np.sin(np.exp(1)*t)
high_idx, low_idx = hl_envelopes_idx(s)

# plot
plt.plot(t,s,label='signal')
plt.plot(t[high_idx], s[high_idx], 'r', label='low')
plt.plot(t[low_idx], s[low_idx], 'g', label='high')

In [ ]:
smoothed_breaths = []
for group in grouped_breaths:
    group = abs(group)
    smoothed = savgol_filter(group, int(0.8* (len(group))), 3)
    plt.plot(smoothed)
    smoothed_breaths.append(smoothed)

Now we will plot out smoothed entropy

In [ ]:
smoothed_breaths_entropy = []
for group in grouped_entropy:
    smoothed = savgol_filter(group, int(0.8* (len(group))), 3)
    plt.plot(smoothed)
    smoothed_breaths_entropy.append(smoothed)

Now we will calculate the area under the curve until the curve of the breath passes it's maximum then hits 70% of it's peak value. 

In [ ]:
curve_area_under = []
for curve in smoothed_breaths:
    max_ind = (curve.argmax())
    max_val = curve[max_ind]
    absolute_val_array = np.abs(curve[max_ind:] - curve.max() * 0.7)
    smallest_difference_index = absolute_val_array.argmin()
    closest_element = curve[max_ind:][smallest_difference_index]
    smallest_difference_index = smallest_difference_index + max_ind
    area_under_curve_cut = curve[:smallest_difference_index].sum()
    curve_area_under.append(area_under_curve_cut)
    

    

In [ ]:
# the future function pending eline approval 
def breath_curve_catch(curve):
    """
    The function is intended for smoothed arrays
    The function takes a smoothed breath array then calculates part of the area under the curve 
    including up to the peak and then to 70% of the peak valye
    """
    max_ind = (curve.argmax())
    max_val = curve[max_ind]
    absolute_val_array = np.abs(curve[max_ind:] - curve.max() * 0.7)
    smallest_difference_index = absolute_val_array.argmin()
    closest_element = curve[max_ind:][smallest_difference_index]
    smallest_difference_index = smallest_difference_index + max_ind
    area_under_curve_cut = curve[:smallest_difference_index].sum()
    return area_under_curve_cut

In [ ]:
# so now we have an array with all of those areas (to 70% of curve ) for each smoothed breath
curve_area_under

OK, but what area these areas? let's see an example of what curve it cuts then takes the area under

In [ ]:
list_of_numbers_to_show = []
for i in range(len(smoothed_breaths)):
    list_of_numbers_to_show.append(str(i))


breath_to_show = widgets.Dropdown(
    options=list_of_numbers_to_show,
    value='1',
    description='Picked Breaths:',
    disabled=False,
)
display(breath_to_show)

In [ ]:
# let's show an example with a graph to show how it looks
curve_example = smoothed_breaths[int(breath_to_show.value)]
max_ind = (curve_example.argmax())
max_val =  curve_example[max_ind]
absolute_val_array = np.abs(curve_example[max_ind:] - curve.max() * 0.7)
smallest_difference_index = absolute_val_array.argmin()
closest_element = curve_example[max_ind:][smallest_difference_index]
smallest_difference_index = smallest_difference_index + max_ind
area_under_curve_cut = curve_example[:smallest_difference_index].sum()
plt.plot(curve_example, color= 'purple', alpha = 0.7)
plt.plot(curve_example[:smallest_difference_index],color = 'green', alpha = 0.9)

## But what if we want to cut on 70% entropy past the peak instead?

In [ ]:
curve_area_under_entropy = []

for i, entropy_curve in enumerate(smoothed_breaths_entropy):
    max_ind = (entropy_curve.argmax())
    max_val = entropy_curve[max_ind]
    absolute_val_array = np.abs(entropy_curve[max_ind:] - entropy_curve.max() * 0.7)
    smallest_difference_index = absolute_val_array.argmin()
    closest_element = entropy_curve[max_ind:][smallest_difference_index]
    smallest_difference_index = smallest_difference_index + max_ind
    area_under_curve_cut = smoothed_breaths[i][:smallest_difference_index].sum()
    curve_area_under_entropy.append(area_under_curve_cut)

In [ ]:
curve_area_under_entropy

In [ ]:
our_emg_array = processed_data_emg[int(start):(int(start) + len(rms_rolled))]
our_entropy_array = rms_rolled
zippy = zip(seven_line,seven_line[1:])
breath_indeces = []
for val in enumerate(zippy):
    if val[1][0] < val[1][1]:
        print(val[0])
        breath_indeces.append(val[0])

grouped_breaths = np.split(our_emg_array, breath_indeces)
grouped_entropy= np.split(rms_rolled, breath_indeces)
grouped_entropy

## So great, we can do it, however we have to agree that the amount of smoothing to calculate these things is acceptable, and whether we use the entropy or the EMG itself

## We also need to agree on what other data is collected into the final spreadsheet

First we can compare counted 'breaths' to algorithm breaths, then we can compare a sequence of peaks.

In [ ]:
my_count = input()

Now let's compare our peak value sequence



Also we are looking at distance from zero to positive maxima, but the amplitude would be from adjancent low value...
We can look at an absolute value array?

In [ ]:
def find_maxima_in_high_entropy_area(our_array,start=0, end=10000, decision_cutoff='mean'):
    """
    Finds maxima in high entropy areas. You need to have made an rms_rolled variable
    on the entropy areas.
    The function is not yet optimized, but works here in the notebook. 
    """
 
    #rms_rolled= 
    decision_array = hf.zero_one_for_jumps_base(rms_rolled,decision_cutoff)
    if decision_array[0] == 1:
        ups_and_downs = np.logical_xor(decision_array[1:], decision_array[:-1])
        indeces_of_boundaries = np.where(ups_and_downs)[0]
        maxima = []
        boundaries = np.append(
            np.append(np.zeros(1), indeces_of_boundaries),
            np.zeros(1) + len(our_array),
        )
        # print(boundaries)
        boundaries = boundaries.astype(np.int32)
        for slice_start, slice_end in zip(boundaries[::2], boundaries[1::2]):
            #print(slice_start, slice_end)
            beat = our_array[slice_start:slice_end]
            maxima.append(slice_start + np.where(beat == beat.max())[0][0])
        maxima_values = our_array[maxima]
        # print(maxima_values)
        rep_array = np.zeros(len(our_array))
        rep_array[maxima] = np.mean(maxima_values)
        plt.plot(our_array, alpha = 0.7)
        plt.plot(rep_array, alpha = 0.4)
    else: 
        ups_and_downs = np.logical_xor(decision_array[1:], decision_array[:-1])
        indeces_of_boundaries = np.where(ups_and_downs)[0]
        maxima = []
        boundaries = np.append(
            indeces_of_boundaries,
            np.zeros(1) + len(our_array),
        )
        boundaries = boundaries.astype(np.int32)
        for slice_start, slice_end in zip(boundaries[::2], boundaries[1::2]):
            #print(slice_start, slice_end)
            beat = our_array[slice_start:slice_end]
            maxima.append(slice_start + np.where(beat == beat.max())[0][0])
        maxima_values = our_array[maxima]
        #print(maxima_values)
        rep_array = np.zeros(len(our_array))
        rep_array[maxima] = np.mean(maxima_values)
        plt.plot(our_array, alpha = 0.7)
        plt.plot(rep_array, alpha = 0.4)
    return maxima, maxima_values
        

In [ ]:
if time_view == 'Samples':
    our_array = processed_data_emg[int(start):int(end)]
    starter=int(start)
    ender= int(end)
else: 
    our_array = processed_data_emg[int(start_s):int(end_s)]
    starter=int(start_s)
    ender= int(end_s)

maximal , maximal_values = find_maxima_in_high_entropy_area(our_array,start=starter, end=ender, decision_cutoff=decision_cutoff)
# note the x axis will be from zero counting up but represent the difference

There are several decisions that need to be made here. Note the following:
    
    We are looking at maxima on the positive. Maybe we should be looking on both sides of zero? Maybe we should be lokking at an absolute value, and then find the maxima? Also note our cut-off on entropy was a bit arbitraty, and gave us an extra breath. These are decisions for the scientific side of a team that need to happen before this interface can go furhter.  

Then we have to decide if we do area under curve for the absolute value, or what?

In [ ]:
Now I can save off the info on this run:

In [ ]:
data_now = [file_chosen, number_chosen, time_view, start, end, my_count, max_per_peak_count, maximal, maximal_values, curve_area_under]

In [ ]:
data_now

In [ ]:
big_data_list.append(data_now)
big_data_list

In [ ]:
columns_now = ['file_chosen',
               'number_file',
               'units',
               'start_cut',
               'end_cut',
               'my_hand_count',
               'automated_breath_count',
               'maxima',
               'maxima_values',
               'curve_area_under',
               'curve_area_under_entropy',
               ]

df = pd.DataFrame(big_data_list, columns=columns_now)

In [ ]:
df

In [ ]:
#You probably want to change the name to a timestamp, and save every hour, at least. 
df.to_csv('my_saved_entropy_experiments.csv')